In [165]:
from fontTools.misc.bezierTools import epsilon
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

In [166]:
data = load_breast_cancer()

In [167]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

In [168]:
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
scaler = StandardScaler()
X_train_d = torch.Tensor(scaler.fit_transform(X_train))
X_train_d.to('cuda')
Y_train_d = torch.Tensor(y_train)
Y_train_d.to('cuda')
Y_test_d = torch.Tensor(y_test)
Y_test_d.to('cuda')
X_test_d = torch.Tensor(scaler.transform(X_test))
X_test_d.to('cuda')

tensor([[-0.7930, -0.2138, -0.7313,  ...,  0.1310,  0.6632,  2.6410],
        [-0.8163,  0.1843, -0.8152,  ..., -1.4482,  0.6374, -1.0436],
        [ 0.4269,  0.4751,  0.4895,  ...,  1.5551,  0.1725,  1.2181],
        ...,
        [ 0.2061,  1.5096,  0.1440,  ..., -0.0602, -0.2795, -0.7956],
        [-0.3661,  2.4655, -0.4186,  ..., -0.7523, -0.7863, -1.0653],
        [-0.0756,  0.7635, -0.0276,  ...,  0.8384,  0.2964,  0.5627]],
       device='cuda:0')

In [169]:
X_test_d.dtype

torch.float32

In [170]:
X_train_d.shape

torch.Size([455, 30])

In [171]:
class Mysimplenn:
    def __init__(self,X):
        self.weights = torch.rand(X.shape[1],1,dtype=torch.float32,requires_grad=True)
        self.bias = torch.zeros(1,1,dtype=torch.float32,requires_grad=True)
    def forward(self,X):
         z = torch.matmul(X,self.weights)+self.bias
         y_pred = torch.sigmoid(z)
         return y_pred
    def loss_function(self,y_pred,y):
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
        loss = -(y*torch.log(y_pred)+(1-y)*torch.log(1-y_pred)).mean()
        return loss



In [172]:
learning_rate = 0.1
epochs = 100

In [173]:
model = Mysimplenn(X_train_d)
print(model.weights)

tensor([[0.1783],
        [0.0905],
        [0.4729],
        [0.9496],
        [0.1355],
        [0.3560],
        [0.2044],
        [0.9351],
        [0.8068],
        [0.6740],
        [0.3109],
        [0.7309],
        [0.2440],
        [0.0870],
        [0.5027],
        [0.1848],
        [0.1607],
        [0.7818],
        [0.8453],
        [0.3783],
        [0.0556],
        [0.3094],
        [0.7890],
        [0.7177],
        [0.8468],
        [0.7267],
        [0.3375],
        [0.7654],
        [0.0655],
        [0.2776]], requires_grad=True)


In [176]:
for epoch in range(epochs):
    y_pred = model.forward(X_train_d)
    loss = model.loss_function(y_pred, Y_train_d)
    loss.backward()

    with torch.no_grad():
        model.weights -= learning_rate*model.weights.grad
        model.bias -=learning_rate*model.bias.grad

    model.weights.grad.zero_()
    model.bias.grad.zero_()
    print(f"epoch: {epoch+1}, loss: {loss.item()}")



epoch: 1, loss: 6.427799224853516
epoch: 2, loss: 6.426502704620361
epoch: 3, loss: 6.424448013305664
epoch: 4, loss: 6.42283821105957
epoch: 5, loss: 6.421529293060303
epoch: 6, loss: 6.420452117919922
epoch: 7, loss: 6.419071197509766
epoch: 8, loss: 6.417410373687744
epoch: 9, loss: 6.415934085845947
epoch: 10, loss: 6.4145941734313965
epoch: 11, loss: 6.413266658782959
epoch: 12, loss: 6.411702632904053
epoch: 13, loss: 6.410368919372559
epoch: 14, loss: 6.409030437469482
epoch: 15, loss: 6.407711029052734
epoch: 16, loss: 6.406512260437012
epoch: 17, loss: 6.4052734375
epoch: 18, loss: 6.403932571411133
epoch: 19, loss: 6.402453422546387
epoch: 20, loss: 6.400927543640137
epoch: 21, loss: 6.3992719650268555
epoch: 22, loss: 6.397877216339111
epoch: 23, loss: 6.396062850952148
epoch: 24, loss: 6.394784450531006
epoch: 25, loss: 6.392126083374023
epoch: 26, loss: 6.39064884185791
epoch: 27, loss: 6.388706684112549
epoch: 28, loss: 6.386616230010986
epoch: 29, loss: 6.384928226470947

In [189]:
with torch.no_grad():
    y_pred = model.forward(X_test_d)
    y_pred = (y_pred>0.5).float()
    accuracy_score = (y_pred == Y_test_d).float().mean()
    print(accuracy_score)

tensor(0.5054)
